$$
\def\E{\mathbb{E}}
\def\1{\mathbb{1}}
$$
In questo modello multiperiodale un'azienda fallisce quando alla scadenza del proprio debito il valore degli attivi si trova al di sotto di una soglia fissa K. Nel caso in cui si trovi sopra l'azienda può accedere ad un nuovo finziamento, ovviamente ad uno spread maggiore del precedente qualora gli attivi siano scesi (minore se saliti).
L'azienda rinnova i propri finanziamenti ogni $\tau$ anni, in teoria per sempre. Per motivi numerici è opportuno inserire una scadenza $T >> \tau.$
Ad ogni data di rinnovo $t = i \cdot \tau$ (dove i = 0,...,N), i finanziatori dell'azienda determinano lo spread minimo a cui sono dispositi a prestare soldi dalla seguente relazione:
$$
L(t) = \E\big[L(t)\cdot(1+s(t))\1_{\{A(t+\tau) > K\}} + A(t+\tau)\1_{\{A(t+\tau < K\}}
|\mathcal{F}_t\big]
$$
Lo spread si può quindi determinare come:
$$
s(t) = \frac{1}{L \cdot N(d_2)}\big[ L(t)\cdot \big(\1-N(d_2)\big) - A(t)\cdot\big(1-N(d_1)\big)\big]
$$
dove $L(t)$ è calcolabile partendo dal finanziamento originario in $t = 0$ come:
$$
L(t=i\tau) = \prod_{j=0}^{j=i-1}(1+s(j\tau))L(0)
$$
La valutazione dell'equity potrebbe essere quindi effettuata in modalità forward (ad esempio tramite montecarlo) fino alla scadenza fittizia T attualizzando:
* 0, nel caso in cui esista un A(t) < K
* $max(A(T) - L(T), 0)$ se arriva a scadenza

Questo modello ha una cosenguenza, ovvero che il default non dipende dalla storia del passivo (solo il payoff ne dipende). Vederemo in seguito se possibile cambiare qualche assunzione.


In [59]:
A = 100

In [60]:
K = 50

In [157]:
vol = 0.2

In [158]:
tau = 1

In [159]:
L = 90

In [160]:
T = 50

In [161]:
import numpy as np
import scipy as sp
import scipy.stats
import math

In [162]:
def s(At, Lt):
    d1 = (math.log(At / K) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(At / K) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    Nd2 = sp.stats.norm.cdf(d2)   
    spread = (Lt*(1-Nd2) - At * (1-Nd1)) / (L*Nd2)
    return spread

In [163]:
print s(A,L)

0.000180191679296


In [164]:
def pathGen(runs):
    nsteps = T / tau + 1
    vol2 = vol * vol
    steps = range(nsteps) * tau
    epsilon = np.random.normal(0, 1, (runs, nsteps - 1))
    paths = np.ones((nsteps, runs))
    paths[0,:] = A
    for i in range(nsteps)[1:]:
        paths[i,:] = paths[i-1,:] * np.exp(-0.5*vol2 * tau + vol * math.sqrt(tau) * epsilon[:,i-1])
    return paths

In [165]:
def payoff(path):
    np.seterr('raise')
    # Check for default
    if min(path) < K:
        return 0.
    
    # Compute the spread
    try:
        Lt = L
        for i in range(len(path) - 1):
            spread = s(path[i], Lt)
            Lt = Lt * (1 + spread)
    except:
        return 0
    
    return max(path[-1] - Lt,0)

In [166]:
runs = 10000


In [167]:
def E():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff(paths[:,i])
    return payoffs.mean()
        

In [168]:
print E()

43.3064537366


Questo modello non funziona, dovremmo ottenere un numero vicino a A - L iniziali. Ci sono più motivi:
* è difficile capire qual è il payoff a scadenza: ho messo il max tra A - L e 0 poiché l'equity non può diventare negativa, ma così facendo il problema non è ben specificato. Ci sono path che terminano con A vicino a K, $L(T)$ enorme e nessuno che soffre A - L. Anche togliendo il max dalla condizione finale non sembra risolvere
* L può crescere indiscriminatamente senza portare al fallimento, con problemi (numerici e concettuali) enormi.

E' necessario cambiare le specifiche del modello, inserendo una condizione di fallimento differente Stavo pensando di inserire una condizione sullo spread, ma credo che questo porti alla necessità di risolvere il problema backward.
Come primo tentativo per ovviare a questo problema si può ipotizzare come condizione di fallimento quella in cui gli attivi si trovano al di sotto di una percentuale $X$ dei passivi emessi alla data precedente.

In [169]:
X = 0.4

In [170]:
def s_perc(At, Lt):
    d1 = (math.log(At / (Lt * X)) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(At / (Lt * X)) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    Nd2 = sp.stats.norm.cdf(d2)   
    spread = (Lt*(1-Nd2) - At * (1-Nd1)) / (L*Nd2)
    return spread

Il problema, come nel caso precedente, è quello di trovare un payoff terminale adeguato. Probabilmente la cosa migliore è ipotizzare, per quanto assurdo, che all'ultimo periodo valga il modello uniperiodale alla Merton e valutare la condizione terminale a $T - \tau$ con il valore dell'equity dato dal valore della call. Rispetto a come ho tratto il caso precedente devo quindi inserire un ulteriore funzione per la deteriminazione dello spread e la funzione di pricing dell'equity.

In [171]:
def bs(A, L):
    d1 = (math.log(A / L) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(A / L) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    call_price = A * sp.stats.norm.cdf(d1) - L * sp.stats.norm.cdf(d2)   
    put_price = L * sp.stats.norm.cdf(-d2) - A * sp.stats.norm.cdf(-d1)
    return put_price, call_price

In [172]:
def b1(A, L, s):
    put, call = bs(A, L * (1+s))
    return L * (1 + s) - put

In [173]:
def s_merton(A, L):
    res = sp.optimize.minimize(
    lambda s: (b1(A, L, s) - L) ** 2,
    [0.0],
    bounds=[(-0.1, 10.0)])
    return res.x[0]


In [185]:
def e_merton(A, L, s):
    put, call = bs(A, L * (1+s))
    return call

In [186]:
def payoff_perc(path):
    Lt = L
    for i in range(len(path) - 1):
        if path[i+1] < X * Lt:
            # Default
            return 0
        if i == (len(path) - 2):
            spread = s_merton(path[i], Lt)
            return e_merton(path[i], Lt, spread)
        else:
            spread = s_perc(path[i], Lt)

        Lt = Lt * (1 + spread)
    
    return max(path[-1] - Lt,0)

In [187]:
def E_perc():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff_perc(paths[:,i])
    return payoffs.mean()

In [188]:
print E_perc()

27.9303393188


Anche questo modello non funziona. Il problema credo dipenda dal fatto che in realtà queste specifiche non sono corrette. Ad esempio nel modello di Merton base il caso in cui il forward degli asset è inferiore al passivo non è ammissibile: non esiste nessun ente disposto a finanziare questo tipo di aziena. Sotto l'esempio numerico di un'aziena che ha debito per 100 e forward degli asset parti a 99.9.

In [184]:
print s_merton(99.9, 100)

10.0


Ovvero lo spread massimo possibile del risolutore numerico pari al 1000%. Quindi, ipotizzare che valga Merton alla scadenza non è opportuno vista la possibiltà di emettere nuovo passivo con un attivo il cui valore/forward è inferiore. Direi anche che forse questa possibilità di rifinanziamento non è verosimile: c'è evidentemente da affrontre il modello in modo molto differente.